In [1]:
import torch
import numpy as np

seed = int(input())
np.random.seed(seed)
torch.manual_seed(seed)

NUMBER_OF_EXPERIMENTS = 200

class SimpleNet(torch.nn.Module):
    def __init__(self, activation):
        super().__init__()

        self.activation = activation
        self.fc1 = torch.nn.Linear(1, 1, bias=False)  # one neuron without bias
        self.fc1.weight.data.fill_(1.)  # init weight with 1
        self.fc2 = torch.nn.Linear(1, 1, bias=False)
        self.fc2.weight.data.fill_(1.)
        self.fc3 = torch.nn.Linear(1, 1, bias=False)
        self.fc3.weight.data.fill_(1.)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

    def get_fc1_grad_abs_value(self):
        return torch.abs(self.fc1.weight.grad)

def get_fc1_grad_abs_value(net, x): #  среднее значение градиента в первом слое.
    output = net.forward(x)
    output.backward()  # no loss function. Pretending that we want to minimize output
                       # In our case output is scalar, so we can calculate backward
    fc1_grad = net.get_fc1_grad_abs_value().item()
    net.zero_grad()
    return fc1_grad


activation =  {'ELU': torch.nn.ELU(), 
               'Hardtanh': torch.nn.Hardtanh(),
               'LeakyReLU': torch.nn.LeakyReLU(), 'LogSigmoid': torch.nn.LogSigmoid(),
               'PReLU': torch.nn.PReLU(), 'ReLU': torch.nn.ReLU(), 'ReLU6': torch.nn.ReLU6(),
               'RReLU': torch.nn.RReLU(), 'SELU': torch.nn.SELU(), 'CELU': torch.nn.CELU(),
               'Sigmoid': torch.nn.Sigmoid(), 'Softplus': torch.nn.Softplus(),
               'Softshrink': torch.nn.Softshrink(), 'Softsign': torch.nn.Softsign(),
               'Tanh': torch.nn.Tanh(), 'Tanhshrink': torch.nn.Tanhshrink(),
               'Hardshrink': torch.nn.Hardshrink()}

for key in activation:
    net = SimpleNet(activation=activation[key])
    
    fc1_grads = []
    for x in torch.randn((NUMBER_OF_EXPERIMENTS, 1)):
        fc1_grads.append(get_fc1_grad_abs_value(net, x))
    # print('{0:10}'.format(key), '{0:10}'.format(np.mean(fc1_grads)))
    print('{0:10} - {1}'.format(key,  np.mean(fc1_grads)))



11
ELU        - 0.46825177246239036
Hardtanh   - 0.2953543031180743
LeakyReLU  - 0.3869488415962411
LogSigmoid - 0.2267814244981855
PReLU      - 0.35629074233864233
ReLU       - 0.39361816555727275
ReLU6      - 0.3758565194578841
RReLU      - 0.3941042095749435
SELU       - 0.5512814357271418
CELU       - 0.4496136348252185
Sigmoid    - 0.00696174914824951
Softplus   - 0.2403151054587215
Softshrink - 0.24748901307582855
Softsign   - 0.06543945872574114
Tanh       - 0.16056774505530483
Tanhshrink - 0.02454929255068891
Hardshrink - 0.7263762906193734


In [3]:
import math 
print('w4',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*math.tanh(math.tanh(math.tanh(100))))
print('w3',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1-math.tanh(math.tanh(math.tanh(100)))**2)*math.tanh(math.tanh(100)))
print('w2',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1-math.tanh(math.tanh(math.tanh(100)))**2)*(1-math.tanh(math.tanh(100))**2)*math.tanh(100))
print('w1',(1 - math.tanh(math.tanh(math.tanh(math.tanh(100))))**2)*(1 - math.tanh(math.tanh(math.tanh(100)))**2)*(1 - math.tanh(math.tanh(100))**2)*(1 - math.tanh(100)**2)*100)

w4 0.43614538244454387
w3 0.30412468309754653
w2 0.1677068587693903
w1 0.0


In [7]:
# По ReLU ответ [100, 100, 100, 100]